In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import warnings

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('../input/dataset.csv')
sub = df[['WindNumber', 'Time']]
df.head()

,WindNumber,Time,WindSpeed,Power,RotorSpeed
0,1,2017/11/1 0:20,4.188333,65.750000,5.779167
1,1,2017/11/1 0:30,4.042500,120.700000,8.636500
2,1,2017/11/1 0:40,4.137368,135.157895,8.668421
3,1,2017/11/1 0:50,4.274737,157.315789,8.727895
4,1,2017/11/1 1:00,4.108947,129.894737,8.656842


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497837 entries, 0 to 497836
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   WindNumber  497837 non-null  int64  
 1   Time        497837 non-null  object 
 2   WindSpeed   497837 non-null  float64
 3   Power       497837 non-null  float64
 4   RotorSpeed  497837 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 19.0+ MB


In [4]:
df['Power_overload'] = np.where(df['Power'] > 2000, 1, 0)

In [5]:
def RotorSpeed_process(x):
    if x['WindNumber'] in [1, 2, 3, 4, 6, 7, 8, 9, 10] and x['RotorSpeed'] > 16.8:
        return 1
    elif x['WindNumber'] == 5 and x['RotorSpeed'] > 19:
        return 1
    elif x['WindNumber'] == 11 and x['RotorSpeed'] > 14:
        return 1
    elif x['WindNumber'] == 12 and x['RotorSpeed'] > 17:
        return 1
    else:
        return 0

In [6]:
df['RotorSpeed_overload'] = df.apply(RotorSpeed_process, axis=1)

In [7]:
df.drop(['Time', 'WindNumber'], axis=1, inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497837 entries, 0 to 497836
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   WindSpeed            497837 non-null  float64
 1   Power                497837 non-null  float64
 2   RotorSpeed           497837 non-null  float64
 3   Power_overload       497837 non-null  int32  
 4   RotorSpeed_overload  497837 non-null  int64  
dtypes: float64(3), int32(1), int64(1)
memory usage: 17.1 MB


In [9]:
clf = IsolationForest(n_estimators=100,
                      random_state=2020,
                      contamination=0.37,
                      n_jobs=-1)
clf.fit(df)
y_pred = clf.predict(df)

In [10]:
y_pred = np.where(y_pred == -1, 1, 0)

In [12]:
np.sum(y_pred)

184200

In [11]:
sub['label'] = y_pred
sub.to_csv('../sub/baseline.csv', index=False)